In [ ]:
import tkinter as tk
from tkinter import messagebox
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

df = pd.read_csv("lung_cancer_info.csv")

df.drop(columns=['YELLOW_FINGERS', 'COUGHING', 'SHORTNESS OF BREATH', 'SWALLOWING DIFFICULTY', 'CHEST PAIN'], inplace=True)
df['LUNG_CANCER'] = LabelEncoder().fit_transform(df['LUNG_CANCER'])
df['GENDER'] = LabelEncoder().fit_transform(df['GENDER'])

Features = df.drop(columns=['LUNG_CANCER'])
Label = df['LUNG_CANCER']
X = Features.to_numpy()
y = Label.to_numpy().reshape(-1,)

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Train KNN model
knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(X_train_scaled, y_train)

# Train Logistic Regression model
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train_scaled, y_train)

# Create and train Voting Classifier
voting_clf = VotingClassifier(estimators=[
    ('knn', knn),
    ('log_reg', log_reg)
], voting='soft')
voting_clf.fit(X_train_scaled, y_train)

knn_predictions = knn.predict(X_test_scaled)
log_reg_predictions = log_reg.predict(X_test_scaled)
voting_predictions = voting_clf.predict(X_test_scaled)

knn_accuracy = accuracy_score(y_test, knn_predictions)
log_reg_accuracy = accuracy_score(y_test, log_reg_predictions)
voting_accuracy = accuracy_score(y_test, voting_predictions)

print(f"KNN Accuracy: {knn_accuracy * 100:.2f}%")
print(f"Logistic Regression Accuracy: {log_reg_accuracy * 100:.2f}%")
print(f"Voting Classifier Accuracy: {voting_accuracy * 100:.2f}%")

root = tk.Tk()
root.title("Lung Cancer Prediction")
root.geometry("400x600")
root.configure(bg="lightblue")

def predict_lung_cancer():
    try:
        user_input = [
            int(entry_gender.get()),  # Gender (1=Male, 0=Female)
            float(entry_age.get()),   # Age
            int(entry_smoking.get()), # Smoking (1=Yes, 0=No)
            int(entry_anxiety.get()), # Anxiety (0-3)
            int(entry_peer_pressure.get()), # Peer Pressure (1=Yes, 0=No)
            int(entry_chronic_disease.get()), # Chronic Disease (1=Yes, 0=No)
            int(entry_fatigue.get()), # Fatigue (0-3)
            int(entry_allergy.get()), # Allergy (1=Yes, 0=No)
            int(entry_wheezing.get()), # Wheezing (0-3)
            int(entry_alcohol_consuming.get()) # Alcohol Consuming (1=Yes, 0=No)
        ]

        input_array = np.array(user_input).reshape(1, -1)
        input_scaled = scaler.transform(input_array)

        print("User Input:", user_input)
        print("Scaled Input:", input_scaled)

        knn_prediction = knn.predict(input_scaled)[0]
        lg_prediction = log_reg.predict(input_scaled)[0]
        voting_prediction = voting_clf.predict(input_scaled)[0]

        print(f"KNN Prediction: {'Yes' if knn_prediction == 1 else 'No'}")
        print(f"Logistic Regression Prediction: {'Yes' if lg_prediction == 1 else 'No'}")
        print(f"Voting Classifier Prediction: {'Yes' if voting_prediction == 1 else 'No'}")

        final_prediction = "Yes" if voting_prediction == 1 else "No"
        messagebox.showinfo("Prediction Result", f"Lung Cancer Prediction: {final_prediction}")

    except ValueError as e:
        messagebox.showerror("Error", f"Invalid input: {e}")

fields = [
    ("Gender (1=Male, 0=Female):", entry_gender := tk.Entry(root)),
    ("Age:", entry_age := tk.Entry(root)),
    ("Smoking (1=Yes, 0=No):", entry_smoking := tk.Entry(root)),
    ("Anxiety (0-3):", entry_anxiety := tk.Entry(root)),
    ("Peer Pressure (1=Yes, 0=No):", entry_peer_pressure := tk.Entry(root)),
    ("Chronic Disease (1=Yes, 0=No):", entry_chronic_disease := tk.Entry(root)),
    ("Fatigue (0-3):", entry_fatigue := tk.Entry(root)),
    ("Allergy (1=Yes, 0=No):", entry_allergy := tk.Entry(root)),
    ("Wheezing (0-3):", entry_wheezing := tk.Entry(root)),
    ("Alcohol Consuming (1=Yes, 0=No):", entry_alcohol_consuming := tk.Entry(root))
]

for field, entry in fields:
    tk.Label(root, text=field, bg="lightblue").pack(pady=5)
    entry.pack(pady=5, padx=20, fill='x')

# Button to trigger prediction
predict_button = tk.Button(root, text="Predict", command=predict_lung_cancer, bg="darkblue", fg="white")
predict_button.pack(pady=20)

root.mainloop()

KNN Accuracy: 95.74%
Logistic Regression Accuracy: 95.74%
Voting Classifier Accuracy: 95.74%
User Input: [1, 62.0, 1, 2, 1, 1, 2, 1, 2, 1]
Scaled Input: [[ 0.9486833  -0.04790092 -1.05409255  1.02421658 -1.01445858 -0.9486833
   0.65689181 -1.06426174  0.90409696 -1.06426174]]
KNN Prediction: Yes
Logistic Regression Prediction: Yes
Voting Classifier Prediction: Yes
